In [1]:
import sys

import torch
import numpy as np

from attacks import AttackContainer
from basemodels import TorchModelContainer, MnistCnnCW
from datasets import (DATASET_LIST, DataContainer, get_image_list,
                      get_quantitative_list)
from defences import DefenceContainer

%load_ext autoreload
%autoreload 2

In [2]:
print(sys.version)
print(*sys.path, sep='\n')

3.6.9 (default, Nov  7 2019, 10:44:02) 
[GCC 8.3.0]
/home/lukec/.vscode/extensions/ms-python.python-2020.2.64397/pythonFiles
/home/lukec/.vscode/extensions/ms-python.python-2020.2.64397/pythonFiles/lib/python
/usr/lib/python36.zip
/usr/lib/python3.6
/usr/lib/python3.6/lib-dynload

/home/lukec/venv/lib/python3.6/site-packages
/home/lukec/Downloads/jax/build
/home/lukec/.local/lib/python3.6/site-packages
/usr/local/lib/python3.6/dist-packages
/usr/lib/python3/dist-packages
/home/lukec/venv/lib/python3.6/site-packages/IPython/extensions
/home/lukec/.ipython


In [3]:
print('Avaliable image datasets:')
print(get_image_list())
print()
print('Avaliable quantitative datasets:')
print(get_quantitative_list())
print()

Avaliable image datasets:
['MNIST', 'CIFAR10', 'SVHN']

Avaliable quantitative datasets:
['BankNote', 'BreastCancerWisconsin', 'WheatSeed', 'HTRU2']



In [4]:
# 1. choose a dataset
DATA_ROOT = 'data'
BATCH_SIZE = 64
TYPE = 'image'  # image or quantitative
# image: 'MNIST', 'CIFAR10', 'SVHN'
# quantitative: 'BankNote', 'BreastCancerWisconsin', 'WheatSeed', 'HTRU2'
NAME = 'MNIST'

print(f'Starting {NAME} data container...')
IMAGE_DATASET = DATASET_LIST[TYPE][NAME]
dc = DataContainer(IMAGE_DATASET, DATA_ROOT)
dc(BATCH_SIZE)

Starting MNIST data container...
Loading data...
Preparing DataLoaders...
Preparing numpy arrays...
Successfully load data! Time taken:  0m 4.1s


In [5]:
# 2. choose a model
model = MnistCnnCW()

In [6]:
# 3. train/load the model
# train, save, load
mc = TorchModelContainer(model, dc)

Using device: cuda:0


In [7]:
mc.fit(epochs=20)

Learning rate: 0.1
[ 1/20]  0m 7.2s - Train Loss: 0.7040 Acc: 74.7650% - Test Loss: 0.0836 Acc: 97.3200%
[ 2/20]  0m 7.1s - Train Loss: 0.0678 Acc: 97.8517% - Test Loss: 0.0449 Acc: 98.6500%
[ 3/20]  0m 7.1s - Train Loss: 0.0425 Acc: 98.6800% - Test Loss: 0.0545 Acc: 98.1400%
[ 4/20]  0m 7.1s - Train Loss: 0.0327 Acc: 98.9917% - Test Loss: 0.0355 Acc: 98.8400%
[ 5/20]  0m 7.2s - Train Loss: 0.0241 Acc: 99.2367% - Test Loss: 0.0281 Acc: 98.9900%
[ 6/20]  0m 7.3s - Train Loss: 0.0183 Acc: 99.4250% - Test Loss: 0.0454 Acc: 98.6900%
[ 7/20]  0m 7.3s - Train Loss: 0.0146 Acc: 99.5267% - Test Loss: 0.0290 Acc: 99.0200%
[ 8/20]  0m 7.1s - Train Loss: 0.0118 Acc: 99.6417% - Test Loss: 0.0359 Acc: 98.8500%
[ 9/20]  0m 7.2s - Train Loss: 0.0097 Acc: 99.6750% - Test Loss: 0.0293 Acc: 99.1300%
[10/20]  0m 7.1s - Train Loss: 0.0092 Acc: 99.6867% - Test Loss: 0.0420 Acc: 98.8700%
[11/20]  0m 7.1s - Train Loss: 0.0093 Acc: 99.6967% - Test Loss: 0.0348 Acc: 98.9900%
[12/20]  0m 7.1s - Train Loss: 0.00

In [8]:
x = torch.rand(2, 1, 28, 28)
mc.pred(x, require_output=True)
x = torch.rand(1, 28, 28)
mc.pred_one(x, require_output=True)

(array(8),
 array([-3.5205455, -2.7884448, -1.6480901, -2.0728738, -4.0678997,
        -3.3718133, -2.9590018, -3.124794 , -0.870921 , -3.6954603],
       dtype=float32))

In [10]:
x = np.random.rand(5, 1, 28, 28).astype(np.float32)
mc.pred(x, require_output=False)

array([7, 0, 3, 4, 8])

In [9]:
mc.save('MNIST_CNN1')

Successfully saved model to "MNIST_CNN1.pt"


In [11]:
mc.load('MNIST_CNN1.pt')

Successfully loaded model from "MNIST_CNN1.pt"


In [3]:
import torch
m = torch.nn.Sigmoid()
loss = torch.nn.BCELoss()
input = torch.randn(3, requires_grad=True)
print(input)
target = torch.empty(3).random_(2)
print(target)
output = loss(m(input), target)
output.backward()
print(input)

tensor([-0.2065, -1.4782, -1.9884], requires_grad=True)
tensor([1., 0., 0.])
tensor([-0.2065, -1.4782, -1.9884], requires_grad=True)
